<a href="https://colab.research.google.com/github/presentchris/RL/blob/main/Reinforcement_Learning_Mario_Game_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install gym-super-mario-bros==7.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=535721 sha256=a966fbcb71b9ec3d202f47cf7864a2a3b3a134f7ee67d2c4431315fc5898261f
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py


In [ ]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy

# Gym은 강화학습을 위한 OpenAI 툴킷
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# OpenAI Gym을 위한 NES 에뮬레이터
from nes_py.wrappers import JoypadSpace

# OpenAI Gym에서의 슈퍼마리오 환경 세팅
import gym_super_mario_bros

##환경 초기화하기

- 환경(Environment) : 에이전트가 상호작용하며 스스로 배우는 세계

- 상태(State) s : 환경의 현재 특성을 상태를 통해 나타냄. 환경이 있을 수 있는 모든 가능한 상태 집합을 상태 공간 이라고 함.

- 행동(Action) a : 에이전트가 환경에 어떻게 응답하는지 행동을 통해 나타냄. 가능한 모든 행동의 집합을 행동 공간 이라고 함.

---

- 마리오 게임에서 환경은 튜브, 버섯, 그 이외 다른 여러 요소들로 구성됨.

- 마리오가 행동을 취하면, 환경은 변경된 (다음)상태, 포상 그리고 다른 정보들로 응답함.

In [ ]:
# 슈퍼 마리오 환경 초기화
if gym.__version__ < '0.26':
  env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", new_step_api = True)
else:
  env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", render_mode = 'rgb', apply_api_compatibility = True)


# 상태 공간을 2가지로 제한하기
# 0 = 오른쪽으로 걷기
# 1 = 오른쪽으로 점프하기
env = JoypadSpace(env, [["right"], ["right", "A"]])

env.reset()
next_state, reward, done, trunc, info = env.step(action = 0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


(240, 256, 3),
 0.0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}


##환경 전처리 과정 거치기

다음 상태(next_state)에서 환경 데이터가 에이전트로 반환됨. 앞서 살펴 보았듯이, 각각의 상태는 [240, 256, 3]의 배열로 나타내고 있음.
종종 상태가 제공하는 것은 에이전트가 필요로 하는 것보다 더 많은 정보임. 예를 들어, 마리오의 행동은 파이프의 색깔이나 하늘의 색깔에 좌우되지 않음.

아래에 설명할 클래스들은 환경 데이터를 에이전트에 보내기 전 단계에서 전처리 과정에 사용할 래퍼(Wrappers)임.

GrayScaleObservation은 RGB 이미지를 흑백 이미지로 바꾸는 일반적인 래퍼임. GrayScaleObservation 클래스를 사용하면 유용한 정보를 잃지 않고 상태의 크기를 줄일 수 있음. GrayScaleObservation을 적용하면 각각의 상태 크기는 [240, 256, 1]이 됨.

ResizeObservation은 각각의 상태(Observation)를 정사각형 이미지로 다운 샘플링함. 이 래퍼를 적용하면 각각 상태의 크기는 [84, 84, 1]이 됨.

SkipFrame은 gym.Wrapper로부터 상속을 받은 사용자 지정 클래스이고, step() 함수를 구현함. 왜냐하면 연속되는 프레임은 큰 차이가 없기 때문에 n개의 중간 프레임을 큰 정보의 손실 없이 건너뛸 수 있기 때문. n번째 프레임은 건너뛴 각 프레임에 걸쳐 누적된 포상을 집계함.

FrameStack은 환경의 연속 프레임을 단일 관찰 지점으로 바꾸어 학습 모델에 제공할 수 있는 래퍼임. 이렇게 하면 마리오가 착지 중이었는지 또는 점프 중이었는지 이전 몇 프레임의 움직임 방향에 따라 확인할 수 있음.

In [ ]:
class SkipFrame(gym.Wrapper):
  def __init__(self, env, skip):
    # 모든 'skip' 프레임만 반환
    super().__init__(env)
    self._skip = skip

  def step(self, action):
    # 행동을 반복하고 포상을 더함
    total_reward = 0.0
    for i in range(self._skip):
      # 포상을 누적하고 동일한 작업을 반복함
      obs, reward, done, trunk, info = self.env.step(action)
      total_reward += reward
      if done:
        break

    return obs, total_reward, done, trunk, info


class GrayScaleObservation(gym.ObservationWrapper):
  def __init__(self, env):
    super().__init__(env)
    obs_shape = self.observation_space.shape[:2]
    self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.unit18)

  def permute_orientation(self, observation):
    # [H, W, C] 배열을 [C, H, W] 텐서로 바꿈
    observation = np.transpose(observation, (2, 0, 1))
    observation = torch.tensor(observation.copy(), dtype = torch.float)
    return observation

  def observation(self, observation):
    observation = self.permute_orientation(observation)
    transform = T.Grayscale()
    observation = transform(observation)
    return observation


class GrayScaleObservation(gym.ObservationWrapper):
  def __init__(self, env):
    super().__init__(env)
    obs_shape = self.observation_space.shape[:2]
    self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

  def permute_orientation(self, observation):
    # [H, W, C] 배열을 [C, H, W] 텐서로 바꿈
    observation = np.transpose(observation, (2, 0, 1))
    observation = torch.tensor(observation.copy(), dtype = torch.float)
    return observation

  def ovservation(self, observation):
    observation = self.permute_orientation(observation)
    transform = T.Grayscale()
    observation = transform(observation)
    return observation


class ResizeObservation(gym.ObservationWrapper):
  def __init__(self, env, shape):
    super().__init__(env)
    if isinstance(shape, int):
      self.shape = (shape, shape)
    else:
      self.shape = tuple(shape)

    obs_shape = self.shape + self.observation_space.shape[2:]
    self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

  def observation(self, observation):
    transforms = T.Compose(
        [T.Resize(self.shape), T.Normalize(0, 255)]
    )
    observation = transforms(observation).squeeze(0)
    return observation


# 래퍼를 환경에 적용함
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
if gym.__version__ < '0.26':
  env = FrameStack(env, num_stack=4, new_step_api = True)
else:
  env = FrameStack(env, num_stack=4)

래퍼를 환경에 적용한 후, 최종 래핑 상태는 4개의 연속된 흑백 프레임으로 구성됨. 마리오가 행동을 할 때마다 환경은 이 구조의 상태로 응답. 구조는 [84, 84, 4] 크기의 3차원 배열로 구성되어 있음.

##에이전트(Agent)
Mario라는 클래스를 이 게임의 에이전트로 생성함. 마리오는 다음과 같은 기능을 할 수 있어야 함.

- 행동(Act)은 (환경의) 현재 상태를 기반으로 최적의 행동 정책에 따라 정해짐
- 경험을 기억(Remember) 하는 것. 경험은 (현재 상태, 현재 행동, 포상, 다음 상태)로 이루어져 있음. 마리오는 그의 행동 정책을 업데이트 하기 위해 캐시(caches)를 한 다음, 그의 경험을 리콜(recalls)함.
- 학습(Learn)을 통해 시간이 지남에 따라 더 나은 행동 정책을 택함.


In [ ]:
class Mario:
  def __init__():
    pass

  def act(self, state):
    # 상태가 주어지면, 입실론-그리디 행동(epsilon-greedy action)을 선택해야 함
    pass

  def cache(self, experience):
    # 메모리에 경험을 추가함
    pass

  def recall(self):
    # 메모리로부터 경험을 샘플링함
    pass

  def learn(self):
    # 일련의 경험들로 실시간 행동 가치(online action value) (Q) 함수를 업데이트함
    pass

이 섹션에서는 마리오 클래스의 매개변수를 채우고, 마리오 클래스의 함수들을 정의함.

##행동하기(Act)

주어진 상태에 대해, 에이전트는 최적의 행동을 이용할 것인지 임의의 행동을 선택하여 분석할 것인지 선택할 수 있음.

마리오는 임의의 행동을 선택했을 때 self.exploration_rate를 활용. 최적의 행동을 이용한다고 했을 때, 그는 최적의 행동을 수행하기 위해 (학습하기(Learn) 섹션에서 구현된) MarioNet이 필요함.

---

- 최적의 행동 - 가치 함수(Action-Value function)  Q*(s,a) : 상태 s에서 시작하면 예상되는 리턴을 반환하고, 임의의 행동 a를 선택함. 그리고 각각의 미래의 단계에서 포상의 합을 극대화하는 행동을 선택하도록 함. Q는 상태에서 행동의 "품질"을 나타냄. 이 함수를 근사시키려고 함.

- 포상(Reward) r : 포상은 환경에서 에이전트로 전달되는 핵심 피트백임. 에이전트가 학습하고 향후 행동을 변경하도록 유도하는 것. 여러 시간 단계에 걸친 포상의 합을 리턴(Return)이라고 함.

In [ ]:
class Mario:
  def __init(self, state_dim, action_dim, save_dir):
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.save_dir = save_dir

    self.device = "cuda" if torch.cuda.is_available() else "cpu"

    # 마리오의 DNN은 최적의 행동을 예측함 -이는 학습하기 섹션에서 구현
    self.net = MarioNet(self.state_dim, self.action_dim).float()
    self.net = self.net.to(device = self.device)

    self.exploration_rate = 1
    self.exploration_rate_decay = 0.99999975
    self.exploration_rate_min = 0.1
    self.curr_step = 0

    self.save_every = 5e5   # MarioNet 저장 사이의 경험 횟수


  def act(self, state):
    """
    주어진 상태에서, 입실론-그리디 행동(epsilon_greedy action)을 선택하고, 스텝의 값을 업데이트 함.

    입력값 : state (''LazyFrame''): 현재 상태에서의 단일 상태(observation)값을 말함. 차원은 (state_dim)임.
    출력값 : ''action_idx'' (int): Mario가 수행할 행동을 나타내는 정수 값.
    """

    # 임의의 행동을 선택하기
    if np.random.rand() < self.exploration_rate:
      action_idx = np.random.randint(self.action_dim)

    # 최적의 행동을 이용하기
    else:
      state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
      state = torch.tensor(state, device = self.device).unsqueeze(0)
      action_values = self.net(state, model = "online")
      action_idx = torch.argmax(action_values, axis=1).item()

    # exploration_rate 감소하기
    self.exploration_rate *= self.exploration_rate_decay
    self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

    # 스텝 수 증가하기
    self.curr_step += 1
    return action_idx

##캐시(Cache)와 리콜(Recall)하기

이 두가지 함수는 마리오의 "메모리" 프로세스 역할을 함.

cache(): 마리오가 행동을 할 때마다, 그는 경험을 그의 메모리에 저장함. 그의 경험에는 현재 상태에 따른 수행된 행동, 행동으로부터 얻은 포상, 다음 상태, 그리고 게임 완료 여부가 포함됨.


recall(): Mario는 자신의 기억에서 무작위로 일련의 경험을 샘플링하여 게임을 학습하는 데 사용함.

Copyright : https://github.com/PyTorchKorea/tutorials-kr

https://github.com/PyTorchKorea/tutorials-kr/blob/master/intermediate_source/mario_rl_tutorial.py